In [55]:
import psycopg2
import pandas as pd

In [96]:
conn = psycopg2.connect(database="accountdb", user="chiubj", 
                        password="bunnygood", host="128.110.25.99", 
                        port="5432")
cur  = conn.cursor()


In [163]:
orders = pd.read_csv('status/orders.csv')
trades = pd.read_csv('status/trades.csv')

In [ ]:

for order_index in range(len(orders)):
    record = [str(orders.iloc[order_index]['交易員代碼']), orders.iloc[order_index]['委託書號'], orders.iloc[order_index]['交易別'], orders.iloc[order_index]['委託日期'], orders.iloc[order_index]['委託時間'],
    str(orders.iloc[order_index]['代碼']), orders.iloc[order_index]['委託種類'], orders.iloc[order_index]['方向'], int(orders.iloc[order_index]['委託部位']), 
    float(orders.iloc[order_index]['委託價格']), orders.iloc[order_index]['狀態'], str(orders.iloc[order_index]['策略別'])]

    table_columns = '(trader_id, order_id, security_type, order_date, order_time, code, order_type, action, order_qty, order_price, status, strategy)'
    postgres_insert_query = f"""INSERT INTO accountdb.dealer.orders {table_columns} VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
    cur.execute(postgres_insert_query, record)

    conn.commit()


In [165]:

for trade_index in range(len(trades)):
    record = [str(trades.iloc[trade_index]['交易員代碼']), trades.iloc[trade_index]['委託書號'], trades.iloc[trade_index]['交易別'], trades.iloc[trade_index]['成交日期'], trades.iloc[trade_index]['成交時間'],
    str(trades.iloc[trade_index]['代碼']), trades.iloc[trade_index]['委託種類'], trades.iloc[trade_index]['方向'], int(trades.iloc[trade_index]['成交部位']), 
    float(trades.iloc[trade_index]['成交價格']), trades.iloc[trade_index]['狀態'], str(trades.iloc[trade_index]['策略別']), trades.iloc[trade_index]['成交序號 (seqno)']]

    table_columns = '(trader_id, order_id, security_type, trade_date, trade_time, code, order_type, action, qty, price, status, strategy, seqno)'
    postgres_insert_query = f"""INSERT INTO accountdb.dealer.trades {table_columns} VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
    cur.execute(postgres_insert_query, record)

    conn.commit()


In [141]:
def postgre_query(query):
    cur.execute(query)
    conn.commit()

In [145]:
postgre_query("ALTER TABLE accountdb.dealer.trades ADD order_type char(3);")

In [125]:
table_column = 'code'
postgres_delete_query = f"""DELETE FROM accountdb.dealer.orders where {table_column} = %s;"""
delete_params = ('6414',)
cur.execute(postgres_delete_query, delete_params)
conn.commit()

In [67]:
cur.close()
conn.close()


create_table_query = '''CREATE TABLE accountdb.dealer.orders(
        trader_id varchar(10),
        broker_id varchar(7),
        account_id varchar(7),
        strategy INT,
        order_id varchar(10),
        security_type char(1), -- S, F
        order_date date,
        order_time time,
        code varchar(10),
        action char(1), -- B, S
        order_price double precision,
        order_qty int,
        order_type varchar(3), -- ROD, IOC, FOK
        price_type varchar(3), -- LMT, MKT, MOP
        status varchar(10),
        CONSTRAINT fk_strategy FOREIGN KEY(strategy) REFERENCES dealer.strategy(id)
    );'''


CREATE TABLE accountdb.dealer.trades(

    trader_id varchar(10),

    broker_id varchar(7),

    account_id varchar(7),

    strategy INT,

    order_id varchar(10),

    seqno varchar(12),

    security_type char(1), -- S, F

    trade_date date,

    trade_time time,

    code varchar(10),

    action char(1), -- B, S

    price double precision,

    qty int,

    status varchar(10),

    CONSTRAINT fk_strategy FOREIGN KEY(strategy) REFERENCES dealer.strategy(id)

);

cursor.execute(create_table_query)
conn.commit()

cursor.close()
conn.close()


cur = conn.cursor()
cur.execute("ROLLBACK")
conn.commit()

